# Import

## Modules

In [1]:
import pandas_datareader as pdr
import pandas as pd
import numpy as np
from datetime import datetime
import scipy.optimize as sco
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import itertools  
import re
from datetime import datetime, timedelta

In [2]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.palettes import Dark2_5 as palette
from bokeh.models import Legend

## Data

In [3]:
portfolio = ["WES.AX", 'IVV.AX', 'QBE.AX', 'CUV.AX', 'ALT.AX']

metrics = ["High", "Low", "Open", "Close", "Adj Close", "Volume"]
columns = pd.MultiIndex.from_product([portfolio, metrics])

n_securities = len(portfolio)

source = 'yahoo'
start = '2016-01-01'
end = '{0}-{1}-{2}'.format(datetime.now().year, datetime.now().month, datetime.now().day)

stocks_df = pd.DataFrame(columns=columns, index=pd.date_range(start=start, end=end, freq="D"))

data = pd.DataFrame()
stocks_dict = {}

#Retrieve stock price data and save just the dividend adjusted closing prices

for share in portfolio:
        data[share] = pdr.DataReader(share, data_source=source, start=start, end=end)['Adj Close']
        temp = pdr.DataReader(share, data_source=source, start=start, end=end)[metrics]
        stocks_df[share] = temp
        
stocks_df.dropna(how="all", inplace=True)

# Preprocessing

In [4]:
data = stocks_df.loc[:, pd.MultiIndex.from_product([portfolio, ["Close"]])]

In [5]:
def build_metrics(data, portfolio, metric):

    default_days = [15, 30, 90, 180, 365, 730]
    
    for share in portfolio:
        data.loc[:, (share, "Pct Change")] = data.loc[:, (share, metric)].pct_change()
        data.loc[:, (share, "Cum Sum Pct Change")] = data.loc[:, (share, "Pct Change")].cumsum()
        for days in default_days:
            data.loc[:, (share, "Rolling mean " + str(days))] = data.loc[:, (share, "Cum Sum Pct Change")].rolling(days).mean()

    data.sort_index(axis=1, inplace=True)
    return data

In [6]:
# Todo: Plot this!

def get_trade_points(data, portfolio, short_term_metric='Rolling mean 15', long_term_metric='Rolling mean 90'):
    
    for share in portfolio:
        difference = data.loc[:, (share, short_term_metric)].values - data.loc[:, (share, long_term_metric)].values
        gradient = np.gradient(difference)

        zero_indices = np.argwhere(np.nan_to_num(np.diff(np.sign(difference)))).flatten()
        mask = data.index[zero_indices]

        neg_grad_indices = np.intersect1d(zero_indices, np.where(gradient < 0)[0])
        mask_sell = data.index[neg_grad_indices]

        pos_grad_indices = np.intersect1d(zero_indices, np.where(gradient > 0)[0])
        mask_buy = data.index[pos_grad_indices]
        
        data.loc[mask_sell, (share, "Trade")] = "Sell"
        data.loc[mask_buy, (share, "Trade")] = "Buy"

        data.loc[:, (share, "Trade")].fillna("Hold", inplace=True)
    data.sort_index(axis=1, inplace=True)
    
    return data

In [7]:
data = build_metrics(data, portfolio, "Close")

In [8]:
data = get_trade_points(data, portfolio)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in less
  if sys.path[0] == '':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in greater
  from ipykernel import kernelapp as app


In [17]:
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook

linewidth = 3

x = data.index
y = x

p = figure(plot_width = 900, plot_height = 600, title = 'Cumulative Sum of Percentage Change', 
               x_axis_label = 'Date [Days]', x_axis_type='datetime', y_axis_label = 'Values')

# To show in notebook
output_notebook()


p.outline_line_color = None

p.xgrid.visible = False
p.ygrid.visible = False

p.xgrid.visible = False
p.ygrid.visible = False

p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks

p.xaxis.axis_line_width = 0
p.xaxis.axis_line_color = None

p.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
p.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks

p.yaxis.axis_line_width = 0
p.yaxis.axis_line_color = None

cum_sum_pct_change = p.line(x, y, line_width=linewidth, color="blue", legend="Cum Sum Pct Change", alpha=0.25, line_dash="solid", muted_alpha=0)
rol_mean_15 = p.line(x, y, line_width=linewidth, color="red", legend="Rolling mean 15", alpha=0.75, muted_alpha=0)
rol_mean_90 = p.line(x, y, line_width=linewidth, color="green", legend="Rolling mean 90", alpha=0.75, muted_alpha=0)
buy = p.circle(x, y, size=8, color="blue", alpha=1, fill_color="white", legend="Buy", muted_alpha=0)    
sell = p.circle(x, y, size=8, color="red", alpha=1, fill_color="white", legend="Sell", muted_alpha=0)    
    
def update(feature, metric="Cum Sum Pct Change"):
    
    cum_sum_pct_change.data_source.data['y'] = data.loc[:, (feature, metric)].values
    rol_mean_15.data_source.data['y'] = data.loc[:, (feature, "Rolling mean 15")].values
    rol_mean_90.data_source.data['y'] = data.loc[:, (feature, "Rolling mean 90")].values
    
    mask_buy = data.loc[:, (feature, "Trade")] == "Buy"
    buy.data_source.data['x'] = data[mask_buy].index
    buy.data_source.data['y'] = data.loc[mask_buy, (feature, metric)].values
    
    mask_sell = data.loc[:, (feature, "Trade")] == "Sell"
    sell.data_source.data['x'] = data[mask_sell].index
    sell.data_source.data['y'] = data.loc[mask_sell, (feature, metric)].values

    push_notebook()
    
p.legend.location = "top_left"

# Click in the legend to remove the corresponding line
p.legend.click_policy="mute"

show(p, notebook_handle=True)

interact(update, feature=portfolio)

Loading BokehJS ...

interactive(children=(Dropdown(description='feature', options=('WES.AX', 'IVV.AX', 'QBE.AX', 'CUV.AX', 'ALT.AX…

<function __main__.update(feature, metric='Cum Sum Pct Change')>

In [12]:
linewidth = 3
metric = "Cum Sum Pct Change"
# create a color iterator
colors = itertools.cycle(palette) 
rolling_columns = [days for days in data[share].columns.values.squeeze() if re.search("Rolling mean \d+", days)]
rolling_columns = ['Rolling mean 15', 'Rolling mean 90']

for share in portfolio:
    p = figure(plot_width = 900, plot_height = 600, title = '{}'.format(share), 
               x_axis_label = 'Date [Days]', x_axis_type='datetime', y_axis_label = 'Values')

    p.outline_line_color = None

    p.xgrid.visible = False
    p.ygrid.visible = False

    p.xgrid.visible = False
    p.ygrid.visible = False

    p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks

    p.xaxis.axis_line_width = 0
    p.xaxis.axis_line_color = None

    p.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
    p.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks

    p.yaxis.axis_line_width = 0
    p.yaxis.axis_line_color = None
    
    # muted_alpha controls the opacity of the line, when muted. muted_color controls the color

    p.line(data.loc[:, (share, metric)].index, data.loc[:, (share, metric)].values, line_width=linewidth, color="blue", legend="Cum Sum Pct Change", alpha=0.25, line_dash="solid")
    for color, rol_col in zip(colors, rolling_columns):
        p.line(data.loc[:, (share, metric)].index, data.loc[:, (share, rol_col)].values, line_width=linewidth, color=color, legend=rol_col, alpha=0.75, muted_alpha=0)
    
    mask_buy = data.loc[:, (share, "Trade")] == "Buy"
    p.circle(data[mask_buy].index, data.loc[mask_buy, (share, metric)].values, size=8, color="blue", alpha=1, fill_color="white", legend="Buy", muted_alpha=0)    
    
    mask_sell = data.loc[:, (share, "Trade")] == "Sell"
    p.circle(data[mask_sell].index, data.loc[mask_sell, (share, metric)].values, size=8, color="red", alpha=1, fill_color="white", legend="Sell", muted_alpha=0)    
    
    p.legend.location = "top_left"
    
    # Click in the legend to remove the corresponding line
    p.legend.click_policy="mute"
    # To show in notebook
    output_notebook()

    # Show the plot
    show(p)

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [14]:
from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

x = np.linspace(0, 2*np.pi, 2000)
y = np.sin(x)

p = figure(title="simple line example", plot_height=300, plot_width=600, y_range=(-5,5),
           background_fill_color='#efefef')
r = p.line(x, y, color="#8888cc", line_width=1.5, alpha=0.8)

def update(f, w=1, A=1, phi=0):
    if   f == "sin": func = np.sin
    elif f == "cos": func = np.cos
    r.data_source.data['y'] = A * func(w * x + phi)
    push_notebook()

show(p, notebook_handle=True)

interact(update, f=["sin", "cos"], w=(0,50), A=(1,10), phi=(0, 20, 0.1))

Loading BokehJS ...

interactive(children=(Dropdown(description='f', options=('sin', 'cos'), value='sin'), IntSlider(value=1, descr…

<function __main__.update(f, w=1, A=1, phi=0)>